# 1. Prepare your video dataset

A video of a person performing sing language.
```
my_video_folder
├───Hksl_bear
│   ├───vid_01.mp4
│   ├───vid_02.mp4
│   ...
├───Hksl_bicycle
│   ├───vid_01.mp4
│   ├───vid_02.mp4
│   ...
├───Hksl_carrot
│   ...
├───Hksl_chef
│   ...
.
.
.
```

(or use our [hksl_jsl_skeletons.zip](https://storage.googleapis.com/bit-studio-static/jobs/shuwa/hksl_jsl_skeletons.zip) ,extract to root directory and skip to step 3)

# 2. Preprocess data for training.

At root dir run  

```
python -m  scripts.video_to_skeleton my_video_folder skeleton_dir
  
```
to create skeleton file for training.
*** 


# 3. Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%%capture
from tensorflow.keras import backend as K
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import *
import tensorflow as tf
import pickle
import gin
from tqdm import tqdm
import gc
from pathlib import Path
import glob
import cv2
import matplotlib.pyplot as plt
import os
import random
import math
import numpy as np
gin.enter_interactive_mode()


In [3]:
# move to root dir.
os.chdir("..")
os.getcwd()

'c:\\Users\\Anne\\Downloads\\2025hack\\2025hack'

In [ ]:
from modules import translator
gin.parse_config_file('configs/translator_train.gin')
gin.parse_config_file('configs/utils.gin')

ValueError: No configurable matching 'modules.translator.translator_manager.TranslatorManager'.
  In file "configs/translator.gin", line 4
    modules.translator.translator_manager.TranslatorManager.labels = %LABELS
  In file "configs/translator_train.gin", line 1
    include 'configs/translator.gin'

# 4. Configs

In [ ]:
skeleton_dir = "data/skeletons"
checkpoint = None #"checkpoints/translator/2h20220915.h5"
target_epoch = 100
steps_per_epoch = 500

# online-hard-mining
n_hards = 50

# 5. Update labels

*When training, we use predefined classes; however, in play mode, we use KNN.
We chose this strategy since training directly on the feature head (with triplet loss or contrastive loss) gave us less robustness.*

In [ ]:
h5_glosses = [p.stem for p in Path(skeleton_dir).glob("*.h5")]
LABELS = {}
for i, g in enumerate(h5_glosses):
    LABELS[g] = [i, g]
N_CLASSES = len(LABELS.keys())
print("N_CLASSES", N_CLASSES)
assert N_CLASSES > 1

In [ ]:
with open("configs/labels.gin", "w") as f:
    dump_dict = json.dumps(LABELS, indent=0,separators=(',', ':'))
    f.writelines(f"LABELS = {dump_dict}\n")    
    f.writelines(f"N_CLASSES = {N_CLASSES}")

In [ ]:
gin.parse_config_file('configs/translator_train.gin')
gin.parse_config_file('configs/utils.gin')

# 6. Create model

In [ ]:
model = translator.get_model()
batch_size = model.outputs[0].shape[0]
n_feats = model.outputs[0].shape[1]
n_classes = model.outputs[1].shape[1]
print("batch_size:", batch_size)
print("n_feats:", n_feats)
print("n_classes:", n_classes)

In [ ]:
if checkpoint is not None:
    model.load_weights(checkpoint)

# 7. Data Generator

In [ ]:
train_generator = translator.DataGenerator(skeleton_dir)

In [ ]:
assert len(train_generator.labels_dict) == N_CLASSES

# 8. Train

In [ ]:
optimizer = tf.optimizers.Adam(1e-3)

acc_metrics = tf.keras.metrics.SparseCategoricalAccuracy()
cce = tf.keras.losses.SparseCategoricalCrossentropy(
    reduction=tf.keras.losses.Reduction.NONE, from_logits=True)

In [ ]:
initial_epoch = 0
hards = None


@tf.function
def custom_train_step(inputs, y_true):
    with tf.GradientTape() as tape:
        feats_pred, cls_pred = model(inputs, training=True)

        cls_loss = cce(y_true, cls_pred)

    grads = tape.gradient(cls_loss, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    acc_metrics.update_state(y_true, cls_pred)

    return cls_loss


for ep in range(initial_epoch, target_epoch):
    acc_metrics.reset_states()
    dh = display("", display_id=True)

    for step in range(steps_per_epoch):
        inputs, y_true = train_generator.__getitem__(0, hards)
        cls_loss = custom_train_step(inputs, y_true)
        cls_loss_np = cls_loss.numpy()

        # Online Hard Mining
        hards_b = np.argsort(cls_loss_np)[-n_hards:]
        hards = y_true[hards_b].squeeze().tolist()

        dh.update(f"epoch-{ep:02d} step-{step} cls_loss-{np.mean(cls_loss_np):.4f} acc-{acc_metrics.result().numpy():.4f}")

    if ep % 5 == 0:
        filepath=f"train_ckpts/{ep:02d}_{acc_metrics.result().numpy():.3f}.h5"
        model.save_weights(filepath)